In [28]:
import torch
import pandas as pd
import random
import math
import time

In [ ]:
class perceptron():
    def __init__(self):
        self.weights = None
        self.bias = None

    def sigmoid(self,x):
        return (1/1 + torch.exp(-x))

    def initializaingweights(self,tensorInput):
        if self.weights == None:
            self.weights = torch.rand(size=(tensorInput.shape[1],),requires_grad=True)
        return self.weights

    def initializingBias(self):
        if self.bias == None:
            self.bias =  torch.rand(size=(1,),requires_grad=True)
        return self.bias

    def MSE(self,outputTensor,ActualTensort):
        amount = len(outputTensor)
        SquaredIndiviual = torch.square(outputTensor - ActualTensort)
        summation = torch.sum(SquaredIndiviual)
        return (1/amount) * summation
    
    def sinLoss(self, output:torch.Tensor, target:torch.Tensor) -> torch.Tensor:
        error = output - target
        return torch.mean(torch.sin(200 * error) + 0.5 * error**2)


    def feedforward(self,tensorInput,weightss=None,biss=None,hurstics:bool=False):
        shapes = tensorInput.shape

        if hurstics: 
            weights = weightss
            bias = biss
        else:
            weights = self.initializaingweights(tensorInput)
            bias = self.initializingBias()

        multiplications = torch.mul(tensorInput,weights)

        summation = torch.sum(multiplications,dim=1)

        biasAddition = summation.add(bias)

        result = biasAddition
        
        return result
    


    def optimization(self,loss:torch.Tensor):
        LR = 0.001
        loss.backward()
        with torch.no_grad():
            ResultGradientWRTWeight = self.weights.grad
            ResultGradientWRTBias = self.bias.grad

            self.weights -= LR * (ResultGradientWRTWeight)

            self.bias -= LR * (ResultGradientWRTBias)

        self.weights.grad.zero_()
        self.bias.grad.zero_()
    


    def fittest(self,input:torch.Tensor,actualOutput:torch.Tensor,weights:torch.Tensor,bias:torch.Tensor):
        with torch.no_grad():
            Nuronoutput = self.feedforward(input,weights,bias,hurstics=True)
            LossError = self.sinLoss(Nuronoutput,actualOutput)
        
        return LossError.mean().item()
    


    def GA(self,input:torch.Tensor,actuaOutput:torch.Tensor,Initial_mutation_rate = 0.2,final_mutationRate = 0.05,populationSize = 200,generation = 100):
        
        population = [
            (torch.rand(input.shape[1],) * 2 - 1,torch.rand(1,)* 2 - 1) for _ in range(populationSize)
        ]
        initial_mutationRate = Initial_mutation_rate
        final_mutationRate = final_mutationRate
        

        for Current_gen in range(generation):
            
            CurrentMutation_rate = initial_mutationRate - (initial_mutationRate - final_mutationRate) * (Current_gen/generation)
            
            fittestScores = [
                self.fittest(input,actuaOutput,parameters[0],parameters[1]) for parameters in population
            ]


            
            

            sortedPop = [x for _,x in sorted(zip(fittestScores,population),key=lambda x: x[0])]
            survivedPop = sortedPop[:math.ceil(populationSize * 0.5)]


           
            newPop = survivedPop.copy()
            while len(newPop) < populationSize:
                parentOne,parentTwo = random.sample(survivedPop,2)

                childWeight = (parentOne[0] + parentTwo[0]) / 2
                childBias = (parentOne[1] + parentTwo[1]) / 2

                if random.random() < CurrentMutation_rate:
                    childWeight += torch.randn_like(childWeight) * 0.2
                if random.random() < CurrentMutation_rate:
                    childBias += torch.randn_like(childBias) * 0.2

                newPop.append((childWeight,childBias))

                

            population = newPop

            bestIndex = torch.tensor(fittestScores).argmin().item()
            bestParameters = population[bestIndex]
            bestLoss = fittestScores[bestIndex]
            
            self.weights = bestParameters[0]
            self.bias = bestParameters[1]
        return bestLoss








In [29]:
from sklearn.preprocessing import StandardScaler
SD = StandardScaler()

df = pd.read_csv("./Housing.csv")

df = df.drop(columns=["mainroad","guestroom","basement","hotwaterheating","airconditioning","parking","prefarea","furnishingstatus"],axis=1)
XScaled = SD.fit_transform(df)

inputTensor = XScaled[:,:-1]
outputTensor = XScaled[:,-1]

inputTensor = torch.tensor(inputTensor)
outputTensor = torch.tensor(outputTensor)

In [89]:
GA_times = []
NumOfTrials = 1

for _ in range(NumOfTrials):

    model = perceptron()
    startTime = time.perf_counter()
    lossHurstics = model.GA(inputTensor,outputTensor)

    endTime = time.perf_counter()
    GA_times.append(endTime - startTime)
GA_Average_Time = torch.Tensor(GA_times).mean()

print(f"Generatic Algorthim Average Time: {GA_Average_Time:.4f} Seconds")
print(f"Generatic Algorthim Error: {lossHurstics:.6f}")


print("------" * 13)


Gradient_Times = []
for _ in range(NumOfTrials):
    NotStuck = True
    model = perceptron()
    i = 0
    startTime = time.perf_counter()
    while NotStuck:
        i += 1
        output = model.feedforward(inputTensor)
        PrevsinError = model.sinLoss(output,outputTensor)
        model.optimization(PrevsinError)
        output = model.feedforward(inputTensor)
        sinError = model.sinLoss(output,outputTensor)
        if round(float(PrevsinError),5) == round(float(sinError),5):
            NotStuck = False
        

    endTime = time.perf_counter()
    Gradient_Times.append(endTime - startTime)

GD_Average_Time = torch.Tensor(Gradient_Times).mean()
GA_loss = model.sinLoss(output=output,target=outputTensor)
print(f"Gradient Average Time: {GD_Average_Time:.4f} Seconds")
print(f"Gradient Error: {GA_loss:.6f}")

print("------" * 13)

print(f"DM-GA is faster by {(GA_Average_Time / GD_Average_Time) * 100:.2f}%" if GA_Average_Time < GD_Average_Time else f"GD is faster by {(GD_Average_Time / GA_Average_Time) * 100:.2f}%")
print(f"DM-GA is better convergence by {(lossHurstics / GA_loss) * 100:.2f}%" if lossHurstics < GA_loss else f"GD is better convergence by {(GA_loss / lossHurstics) * 100:.2f}%")




Generatic Algorthim Average Time: 3.5551 Seconds
Generatic Algorthim Error: 0.244104
------------------------------------------------------------------------------
Gradient Average Time: 12.2340 Seconds
Gradient Error: 0.334265
------------------------------------------------------------------------------
DM-GA is faster by 29.06%
DM-GA is better convergence by 73.03%
